In [1]:
%matplotlib inline
import numpy as np
import matplotlib.pyplot as plt
import emcee
import batman
import lightkurve
from lightkurve import KeplerTargetPixelFile
from lightkurve.lightcurve import LightCurve
import astropy.io.fits as fits
from astropy.io import ascii
from scipy import interpolate
import math
import scipy.special

In [2]:
desktop = '/Users/willwaalkes/Desktop/'

def BATMAN_MODEL(Baseline, Rp, t = None):
    #Next we create a TransitParams object to store the physical parameters describing the transit:

    params = batman.TransitParams()
    params.t0 = 0.                       #time of inferior conjunction
    params.per = 39.09432                #period in hours
    params.rp = Rp                       #planet radius (in units of stellar radii)
    params.a = 16.                       #semi-major axis (in units of stellar radii)
    params.inc = 90.                     #orbital inclination (in degrees)
    params.ecc = 0.                      #eccentricity
    params.w = 90.                       #longitude of periastron (in degrees)
    params.u = [0.1956, 0.3700]          #limb darkening coefficients [u1, u2]
    params.limb_dark = "quadratic"       #limb darkening model
        
    m = batman.TransitModel(params, t)    #initializes model
    
    flux = m.light_curve(params)*Baseline        #calculates light curve
    return flux 

highres_times = np.linspace(-18, 2.5, 500)

In [3]:
data_v1_blue = ascii.read(desktop+'Colorado/Research/GJ1132b/GJ1132_Codes/STIS_Codes/erg_visit1_blue.txt')
data_v2_blue = ascii.read(desktop+'Colorado/Research/GJ1132b/GJ1132_Codes/STIS_Codes/erg_visit2_blue.txt')
data_v1_red = ascii.read(desktop+'Colorado/Research/GJ1132b/GJ1132_Codes/STIS_Codes/erg_visit1_red.txt')
data_v2_red = ascii.read(desktop+'Colorado/Research/GJ1132b/GJ1132_Codes/STIS_Codes/erg_visit2_red.txt')

In [4]:
alldata = {}
alldata['V1 Blue'] = data_v1_blue['Flux'],data_v1_blue['Error'],data_v1_blue['Time_hr']
alldata['V1 Red'] = data_v1_red['Flux'],data_v1_red['Error'],data_v1_blue['Time_hr']
alldata['V2 Blue'] = data_v2_blue['Flux'],data_v2_blue['Error'],data_v2_blue['Time_hr']
alldata['V2 Red'] = data_v2_red['Flux'],data_v2_red['Error'],data_v2_blue['Time_hr']

In [5]:
Posteriors = {}
Posterior_bins = {}